# ИД23-1 Маслов АН
## Пример 1. Запись двух DataFrame в один hdf5 файл


In [45]:
%%file rand_csv.py
import random
import time

def rand_csv(length, pos, time_start, time_end, output): #генератор случайно даты и числа
    csv_rows = []
    for i in range(length):
        stime = time.mktime(time.strptime(time_start, '%m/%d/%Y %I:%M %p'))
        etime = time.mktime(time.strptime(time_end, '%m/%d/%Y %I:%M %p'))
        ptime = stime + random.random() * (etime - stime)
        csv_rows.append([time.strftime('%y%m%d%H%M%S', time.localtime(ptime)), random.randint(1, 9999999), pos])
    output.put(csv_rows)

Overwriting rand_csv.py


In [46]:
import rand_csv
import multiprocessing as mp
import random
import pandas as pd
import os

output = mp.Queue()

#29.09.2025 добавлен номер процесса
processes = [mp.Process(target=rand_csv.rand_csv, args=(random.randint(50000,500000), x, "1/1/2008 1:30 PM", "1/1/2025 4:50 AM", output)) for x in range(4)] #создаем 4 процесса
for p in processes:
    p.start()

results = [output.get() for p in processes] #сохраняем результат

for p in processes:
    p.join()

pandas_result = []
for i in results:
    pandas_result = pandas_result + i #форматируем для удобства

#29.09.2025 добавлен номер процесса
resultdata = pd.DataFrame(pandas_result, columns=["datetime", "rand_number", "processor_number"]) #создаем датафрейм
resultdata['datetime'] = pd.to_datetime(resultdata['datetime'], format='%y%m%d%H%M%S')
resultdata = resultdata.sort_values('datetime') #сортируем по дате
resultdata.to_csv("result_data.csv")
resultdata


,datetime,rand_number,processor_number
686228,2008-01-01 13:32:59,4894857,2
654779,2008-01-01 13:42:56,9603697,2
876053,2008-01-01 13:43:18,7365188,2
104317,2008-01-01 13:44:48,4714268,3
875416,2008-01-01 13:44:54,7579588,2
...,...,...,...
1029136,2025-01-01 04:25:17,6559749,2
675882,2025-01-01 04:29:19,8318919,2
247334,2025-01-01 04:41:44,5327785,0
475190,2025-01-01 04:43:06,2161706,0


In [47]:
another_resultdata = pd.read_csv("recipes_sample.csv")
another_resultdata

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [48]:
resultdata.to_hdf('data.hdf5', key='df', mode='w')  
another_resultdata.to_hdf('data.hdf5', key="df2")

C:\Users\alexm\AppData\Local\Temp\ipykernel_16560\1531969755.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['name', 'submitted', 'description'], dtype='object')]

  another_resultdata.to_hdf('data.hdf5', key="df2")


In [49]:
pd.read_hdf('data.hdf5', 'df') 

,datetime,rand_number,processor_number
686228,2008-01-01 13:32:59,4894857,2
654779,2008-01-01 13:42:56,9603697,2
876053,2008-01-01 13:43:18,7365188,2
104317,2008-01-01 13:44:48,4714268,3
875416,2008-01-01 13:44:54,7579588,2
...,...,...,...
1029136,2025-01-01 04:25:17,6559749,2
675882,2025-01-01 04:29:19,8318919,2
247334,2025-01-01 04:41:44,5327785,0
475190,2025-01-01 04:43:06,2161706,0


In [50]:
pd.read_hdf('data.hdf5', 'df2') 

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [51]:
print(os.path.getsize('result_data.csv') + os.path.getsize('recipes_sample.csv'))
print(os.path.getsize('data.hdf5'))

49078032
44441312
